

<h1> <center>Análise de Dados - Cadastro Geral de Empregados e Desempregados - BRASIL - 2019 </center></h1>

Fonte: site basesedosdados.org


Nesta análise foi considerado o uso de trabalhadores negros para soma de pretos e partos, os reabilitados foram renomeados para a categoria 'não deficiente'.E as categorias de escolaridade(Até a 5ª série, 6ª série ao 9º ano) foram renomeadas para 'Fundamental incompleto'.

In [2]:
#Acessando o  Google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Instalando a biblioteca gglpot
!pip install ggplot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.2 MB 5.2 MB/s 


In [4]:
#Importando as bibliotecas
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from plotnine import ggplot, geom_point, aes, stat_smooth, facet_wrap


        Ao acessar os dados do CAGED pelo datalake público(https://basedosdados.org/dataset/br-me-caged) foi feito um filtro pelo ano com SQL e selecionado apenas 2019. Obtivemos uma base final composta por 30.865.840 registros com 40 colunas. e assim para observar a distribuição pelo território nacional deste ano foi feita a extração da quantidade de trabalhadores por estado.





In [5]:
#Concatenando as tabelas, o dataframe df contém dados com os trabalhadores cadastrados no Brasil em 2019)
df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CagedBRFem.csv')
df2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CagedBRMasc.csv')
df = pd.concat([df1,df2])

In [6]:
#Dicionário com as categorias atribuídas aos números
dictSituacao = {1:'Admissao',
                2:'Desligamento'}

dictRacaCor = {1: 'Indigena',
               2: 'Branca',
               4: 'Negra',
               6: 'Amarela',
               8: 'Negra',
               9: 'Nao identificado',
              -1: 'Ignorado'}

dictSexo = {1:'Masculino',
            2:'Feminino'}

dictMes = {
    1 : 'Janeiro',
    2 : 'Fevereiro',
    3 : 'Março',
    4 : 'Abril',
    5 : 'Maio',
    6: 'Junho',
    7 : "Julho",
    8 : 'Agosto',
    9 : 'Setembro',
    10 : 'Outubro',
    11 : 'Novembro',
    12: 'Dezembro'
}

dictIndDef = {
    1:'com deficiencia', 
    0:'sem deficiencia'
}
dictTipdef = {
    1:'Fisica',
    2:'Auditiva',
    3:'Visual',
    4:'Intelectual(Mental)',
    5:'Multipla',
    6:'Nao deficiente',
    0:'Nao deficiente',
    -1:'ignorado' 
}
dictEsc = {
    1:'Analfabeto',
    2:'Fundamental incompleto',
    3:'Fundamental incompleto',
    4:'Fundamental incompleto',
    5:'Fundamental completo',
    6:'Medio incompleto',
    7:'Medio completo',
    8:'Superior incompleto',
    9:'Superior completo',
    10:'Mestrado',
    11:'Doutorado',
    -1:'Ignorado',
    
}  

df = df.replace({'mes': dictMes, 'admitidos_desligados': dictSituacao, 'indicador_portador_deficiencia': dictIndDef, 'tipo_deficiencia': dictTipdef,'sexo' : dictSexo,'raca_cor' : dictRacaCor,'grau_instrucao':dictEsc}) 
df

,mes,sigla_uf,admitidos_desligados,salario_mensal,indicador_portador_deficiencia,tipo_deficiencia,grau_instrucao,idade,sexo,raca_cor
0,Janeiro,AC,Admissao,1077,sem deficiencia,Nao deficiente,Medio completo,20,Feminino,Negra
1,Janeiro,AC,Admissao,1077,sem deficiencia,Nao deficiente,Medio completo,24,Feminino,Negra
2,Janeiro,AC,Admissao,1617,sem deficiencia,Nao deficiente,Medio completo,23,Feminino,Negra
3,Janeiro,AC,Admissao,1500,sem deficiencia,Nao deficiente,Superior completo,31,Feminino,Negra
4,Janeiro,AC,Admissao,1047,sem deficiencia,Nao deficiente,Fundamental completo,18,Feminino,Nao identificado
...,...,...,...,...,...,...,...,...,...,...
18621863,Janeiro,SP,Admissao,1485,sem deficiencia,Nao deficiente,Medio completo,30,Masculino,Negra
18621864,Janeiro,SP,Admissao,1500,sem deficiencia,Nao deficiente,Medio completo,24,Masculino,Nao identificado
18621865,Janeiro,SP,Admissao,1506,sem deficiencia,Nao deficiente,Medio completo,35,Masculino,Branca
18621866,Janeiro,SP,Admissao,1506,sem deficiencia,Nao deficiente,Medio completo,38,Masculino,Branca


In [7]:
#Informações do dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30865840 entries, 0 to 18621867
Data columns (total 10 columns):
 #   Column                          Dtype 
---  ------                          ----- 
 0   mes                             object
 1   sigla_uf                        object
 2   admitidos_desligados            object
 3   salario_mensal                  int64 
 4   indicador_portador_deficiencia  object
 5   tipo_deficiencia                object
 6   grau_instrucao                  object
 7   idade                           int64 
 8   sexo                            object
 9   raca_cor                        object
dtypes: int64(2), object(8)
memory usage: 2.5+ GB


A análise visa fazer um estudo sobre os salários,raça e deficiência dos trabalhadores. Encontramos 4.742.068 trabalhadores com raça/cor não identificada,311.602 trabalhadores com salário informado com 0 e 95.339 trabalhadores com 66 anos ou mais, esses dados foram excluídos e analisamos apenas 25.716.831 registros.


In [8]:
#Identificando os registros de trabalhadores com a raça/cor não identificado 
df['raca_cor'].value_counts()

Branca              13107623
Negra               12783850
Nao identificado     4742068
Amarela               154841
Indigena               77458
Name: raca_cor, dtype: int64

In [9]:
#Selecionando todos os registros que não possuem a raça/cor informada como 'Nao identificado'
df = df.loc[df['raca_cor'] != 'Nao identificado']

In [10]:
#Identificando os salários mensais informados com zero
df.loc[df['salario_mensal'] == 0]

,mes,sigla_uf,admitidos_desligados,salario_mensal,indicador_portador_deficiencia,tipo_deficiencia,grau_instrucao,idade,sexo,raca_cor
46,Janeiro,AC,Admissao,0,sem deficiencia,Nao deficiente,Medio completo,23,Feminino,Negra
256,Janeiro,AP,Desligamento,0,sem deficiencia,Nao deficiente,Superior completo,30,Feminino,Branca
277,Janeiro,AP,Desligamento,0,sem deficiencia,Nao deficiente,Medio completo,34,Feminino,Branca
581,Janeiro,AM,Admissao,0,sem deficiencia,Nao deficiente,Medio completo,23,Feminino,Branca
637,Janeiro,AM,Admissao,0,sem deficiencia,Nao deficiente,Superior completo,35,Feminino,Negra
...,...,...,...,...,...,...,...,...,...,...
18619723,Janeiro,SP,Admissao,0,sem deficiencia,Nao deficiente,Medio completo,34,Masculino,Branca
18619815,Janeiro,SP,Admissao,0,sem deficiencia,Nao deficiente,Fundamental completo,46,Masculino,Negra
18619992,Janeiro,SP,Admissao,0,sem deficiencia,Nao deficiente,Fundamental incompleto,47,Masculino,Negra
18619993,Janeiro,SP,Admissao,0,sem deficiencia,Nao deficiente,Fundamental completo,38,Masculino,Negra


In [11]:
#Selecionando todos os registros de trabalhadores que não possuem salário informado com zero
df = df.loc[df['salario_mensal'] != 0]

In [12]:
#Identificando os registros de trabalhadores que possuem 66 anos ou mais
df.loc[df['idade'] >= 66]


,mes,sigla_uf,admitidos_desligados,salario_mensal,indicador_portador_deficiencia,tipo_deficiencia,grau_instrucao,idade,sexo,raca_cor
205,Janeiro,AP,Admissao,2377,sem deficiencia,Nao deficiente,Superior completo,67,Feminino,Negra
1572,Janeiro,AL,Desligamento,1100,sem deficiencia,Nao deficiente,Medio completo,69,Feminino,Branca
2312,Janeiro,MA,Admissao,1089,sem deficiencia,Nao deficiente,Medio completo,66,Feminino,Negra
2691,Janeiro,MA,Desligamento,1041,sem deficiencia,Nao deficiente,Fundamental incompleto,67,Feminino,Negra
3595,Janeiro,RO,Desligamento,1083,sem deficiencia,Nao deficiente,Medio completo,72,Feminino,Negra
...,...,...,...,...,...,...,...,...,...,...
18621088,Janeiro,SP,Admissao,1489,sem deficiencia,Nao deficiente,Fundamental incompleto,67,Masculino,Branca
18621328,Janeiro,SP,Admissao,10404,sem deficiencia,Nao deficiente,Medio completo,67,Masculino,Branca
18621364,Janeiro,SP,Admissao,1820,sem deficiencia,Nao deficiente,Fundamental incompleto,69,Masculino,Branca
18621464,Janeiro,SP,Admissao,1117,sem deficiencia,Nao deficiente,Fundamental incompleto,68,Masculino,Branca


In [13]:
df = df.loc[df['idade'] <= 65]
df

,mes,sigla_uf,admitidos_desligados,salario_mensal,indicador_portador_deficiencia,tipo_deficiencia,grau_instrucao,idade,sexo,raca_cor
0,Janeiro,AC,Admissao,1077,sem deficiencia,Nao deficiente,Medio completo,20,Feminino,Negra
1,Janeiro,AC,Admissao,1077,sem deficiencia,Nao deficiente,Medio completo,24,Feminino,Negra
2,Janeiro,AC,Admissao,1617,sem deficiencia,Nao deficiente,Medio completo,23,Feminino,Negra
3,Janeiro,AC,Admissao,1500,sem deficiencia,Nao deficiente,Superior completo,31,Feminino,Negra
6,Janeiro,AC,Admissao,1020,sem deficiencia,Nao deficiente,Medio completo,36,Feminino,Negra
...,...,...,...,...,...,...,...,...,...,...
18621862,Janeiro,SP,Admissao,1576,sem deficiencia,Nao deficiente,Medio completo,21,Masculino,Branca
18621863,Janeiro,SP,Admissao,1485,sem deficiencia,Nao deficiente,Medio completo,30,Masculino,Negra
18621865,Janeiro,SP,Admissao,1506,sem deficiencia,Nao deficiente,Medio completo,35,Masculino,Branca
18621866,Janeiro,SP,Admissao,1506,sem deficiencia,Nao deficiente,Medio completo,38,Masculino,Branca


In [14]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/BaseCaged2019.csv')